# what stats need to be done
## Table 2
- apo-holo pairs 
- cryptic pockets
- promiscuous pockets 
- multi-chain pockets
- avg. pocket size 
- avg. # of observed residues

In [3]:
import json
import statistics
import os 
import sys 
sys.path.append('../B-create-dataset')
import filter_utils
import pandas as pd

In [14]:
INPUT_PATH = '../../data/E-add-noncryptic-pockets/ahoj-v2'

with open(f'{INPUT_PATH}/cryptobench/dataset.json') as f:
    dataset = json.load(f)


### (apo-holo pairs) rename to something else, i.e. `APO STRUCTURES` or something alike!!!:

In [5]:
len(dataset.keys())

1107

### No. of cryptic pockets:
QUESTION: How to set `MAX_POCKET_SIMILARITY`? Isn't 75% too much?

In [19]:
MAX_POCKET_SIMILARITY = 0.75

# dictionary of new pockets
merged_pockets = {}

for id, pckts in dataset.items():
    list_of_pocket_definitions = []
    old_pocket_list: [[str]] = []
    for pckt in pckts:
        pckt_def: [str] = pckt['apo_pocket_selection']
        old_pocket_list.append(pckt_def)
    new_pocket_list = []
    j = 0
    while len(old_pocket_list) != len(new_pocket_list):
        if j != 0:
            old_pocket_list = new_pocket_list
        new_pocket_list = []
        merged_lists = []
        for i in range(len(old_pocket_list)):
            if i in merged_lists:
                continue
            for ii in range(i + 1, len(old_pocket_list)):
                fst, snd = (old_pocket_list[i], old_pocket_list[ii]) if len(old_pocket_list[i]) < len(
                    old_pocket_list[ii]) else (old_pocket_list[ii], old_pocket_list[i])
                intersection = set(fst).intersection(snd)
                intersect_len = len(intersection)

                if intersect_len / len(fst) > MAX_POCKET_SIMILARITY or intersect_len / len(snd) > MAX_POCKET_SIMILARITY:
                    merged_lists.append(ii)
                    old_pocket_list[i] = list(set(fst + snd))
            new_pocket_list.append(old_pocket_list[i])
        j += 1
    merged_pockets[id] = old_pocket_list

pockets_count = 0
for id, pckts in merged_pockets.items():
    pockets_count += len(pckts)

print(f'number of cryptic pockets: {pockets_count}')

pockets_count = 0
for id, pckts in dataset.items():
    pockets_count += len(pckts)

print(f'(number of all (even duplicated) pockets: {pockets_count})')


number of cryptic pockets: 1361
(number of all (even duplicated) pockets: 5493)


### No. of promiscuous pockets:

In [20]:
promiscuous_pockets_count = 0
for id, pckts in merged_pockets.items():
    for pckt in pckts:
        ligands = set()
        for pckt_ in dataset[id]:
            pckt_def: [str] = pckt_['apo_pocket_selection']

            if len(set(pckt_def).intersection(pckt)) == len(pckt_def):
                ligands.add(pckt_['ligand'])
        if len(ligands) > 1:
            promiscuous_pockets_count += 1

print(f'number of flexible cryptic pockets: {promiscuous_pockets_count}')


number of flexible cryptic pockets: 371


### No. of multichain pockets:

In [21]:
multiple_chain_pockets = 0
for id, pckts in merged_pockets.items():
    for pckt in pckts:
        number_of_chains = len(set([x.split('_')[0] for x in pckt]))
        if number_of_chains > 1:
            multiple_chain_pockets += 1
print(f'number of cryptic pockets over multiple chains: {multiple_chain_pockets}')


number of cryptic pockets over multiple chains: 197


### avg. pocket size:

In [24]:
lengths = []

for key, value in dataset.items():
    for pocket in value:
        lengths.append(len(pocket['apo_pocket_selection']))

print(f"Average number of residues in pocket - mean: {statistics.mean(lengths)}; variance: {statistics.stdev(lengths)}")


Average number of residues in pocket - mean: 16.608774804296377; variance: 7.216381192241173


### avg. No. of observed residues

In [ ]:
# TODO: wait for computing the sequences for the embedder 
# TODO: this code expects a folder where each file is one protein and the files contains just one row containing the sequence

if False: 
    SEQUENCES_PATH = 'apo-sequences'

    lens = []

    for file in os.listdir(SEQUENCES_PATH):
        with open(f'{SEQUENCES_PATH}/{file}', 'r') as f:
            a = f.readline() 
            lens.append(len(a))

    print(statistics.mean(lens))
    print(statistics.stdev(lens))


## Table 3

In [30]:
FOLD_PATH = f'{INPUT_PATH}/cryptobench/folds'

filenames = [f'{FOLD_PATH}/test.json',
             f'{FOLD_PATH}/train-fold-0.json',
             f'{FOLD_PATH}/train-fold-1.json',
             f'{FOLD_PATH}/train-fold-2.json',
             f'{FOLD_PATH}/train-fold-3.json']
import json

total_len = 0
for data_filepath in filenames:
    with open(data_filepath) as f:
        dataset = json.load(f)
    print(data_filepath.split('/')[-1], len(dataset))
    total_len += len(dataset)

print('total: ', total_len)


test.json 222
train-fold-0.json 219
train-fold-1.json 222
train-fold-2.json 222
train-fold-3.json 222
total:  1107


# TODO
1. recalculate the violin graphs from Fig. 1
2. Calculate the steps in Table 1.
3. remove homomers from noncryptic pockets
4. Think about some statistics for noncryptic pockets

### TABLE 1.

In [10]:
INPUT_PATH = '../../data/A-filter-ahojdb-v2'
INPUT_CSV = f'{INPUT_PATH}/pairs.csv'

fields = ['apo_tm_score', 'apo_tm_score_i', 'apo_pocket_dist', 'apo_RoG', 'holo_RoG', 'apo_UNPovrlp_obs', 'apo_pocket_rms']
ahoj2_df = pd.read_csv(INPUT_CSV, usecols=fields)

In [11]:
# resolution filter:
print(f'resolution filter: {len(ahoj2_df)}')

resolution filter: 4683968


In [31]:
# TODO: redefine the alignment filter according to the criteria in the filter_utils.get_well_defined_pairs(...):
print(f'resolution filter: {len(filter_utils.get_well_defined_pairs(ahoj2_df))}')

resolution filter: 4423220


In [32]:
# pocket RMSD filter:
ahoj2_df = filter_utils.get_well_defined_pairs(ahoj2_df)
filtered_rmsd_df = ahoj2_df[(
            ahoj2_df['apo_pocket_rms'] > filter_utils.MAX_POCKET_RMSD)]
print(f'pocket RMSD filter: {len(filtered_rmsd_df)}')

pocket RMSD filter: 221026


In [15]:
# clustering & selection of representatives

print(f'clustering & selection of representatives: {len(dataset)}')

clustering & selection of representatives: 1107


In [16]:
# additional pockets search

additional_cryptic_pockets = 0
for _, holo_structures in dataset.items():
    additional_cryptic_pockets += len(holo_structures)

print(f'additional pockets search: {additional_cryptic_pockets}')

additional pockets search: 5493


In [27]:
# with open(f'/home/vit/Downloads/cryptobench (2)/auxiliary-data/non-cryptic-pockets/noncryptic-pockets.json') as f:

with open(f'/home/vit/Projects/cryptobench/data/E-add-noncryptic-pockets/ahoj-v2/cryptobench/auxiliary-data/non-cryptic-pockets/noncryptic-pockets.json') as f:
    non_cryptic_dataset = json.load(f)

noncryptic_pockets = 0
for _, holo_structures in non_cryptic_dataset.items():
    noncryptic_pockets += len(holo_structures)

print(f'additional noncryptic pockets search: {noncryptic_pockets}')

additional noncryptic pockets search: 14493
